# Exercise 6: Classical Design of Experiments II (Extended)

In this notebook, we will implement the Haldon sequence as a **space filling** technique for experimental design. 
We will use this implementation and compare it to uniform sampling as another (trivial) space filling DOE technique.

### **Table of Contents**
1. [Halton Sequence](#halton)
2. [Comparison of Space Filling Techniques](#comparison)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

### **1. Halton Sequence** <a class="anchor" id="halton"></a>

A Halton sequence is a low-discrepancy sequence with the property that for all values of $N$, its subsequence $\mathbf{x}_1, ..., \mathbf{x}_N$ has a low **discrepancy**. Roughly said, a low discrepancy is a kind of criterion to express the fact that a sequence fills the segment leaving no gaps. Low-discrepancy sequences are also called quasirandom sequences, due to their common use as a replacement of uniformly distributed random numbers. The *quasi* modifier is used to denote more clearly that the values of a low-discrepancy sequence are neither random nor pseudorandom. Still, such sequences share some properties of random variables.

#### Question:
1. (a) What are advantages of quasirandom numbers over pure random numbers?

   BEGIN SOLUTION
   
   Quasirandom numbers have an advantage over pure random numbers in that they cover the domain of interest quickly and evenly. They have an advantage over purely deterministic methods because they only give high accuracy when the number of samples is preset. In contrast, using quasirandom sequences improves the accuracy as more samples are added, with full reuse of the existing points. On the other hand, quasirandom point sets can have a significantly lower discrepancy for a given number of points than purely random sequences.
   
   END SOLUTION 
   
Halton sequences generalize the one-dimensional van der Corput sequences. A van der Corput sequence is constructed by reversing the base-$n$ representation of the sequence of natural numbers, i.e., $1, 2, 3, \dots$.

For a given base $b = 2, 3, \dots$, the $b$-adic expansion of the positive integer $n \geq 1$ is expressed as

\begin{equation*}
    n = \sum_{j=1}^{T} a_j b^{j-1},
\end{equation*}

where $0 \leq a_j < b$ are the integer coefficients of the expansion. Accordingly, the $n$-th number in the van der Corput sequence is defined through

\begin{equation*}
    \phi_b(n) = \sum_{j=1}^{T} \frac{a_j}{b^{j}}.
\end{equation*}

#### Question:
1. (b) Which numbers are represented through $\phi_2(7)$ and $\phi_3(5)$? Use the above equations to answer this question.

   BEGIN SOLUTION
   
   The $b=2$-adic expansion of $n=7$ is given through
   \begin{equation*}
       7 = 1 \cdot 2^0 + 1 \cdot 2^1  + 1 \cdot 2^2,
   \end{equation*}
   such that $a_1 = a_2 = a_3 = 1$. Accordingly, we obtain
   \begin{equation*}
       \phi_2(7) = \frac{1}{2^1} + \frac{1}{2^2} + \frac{1}{2^3} = \frac{1}{2} + \frac{1}{4} + \frac{1}{8} = \frac{7}{8} = 0.875.
   \end{equation*}
   
   The $b=3$-adic expansion of $n=5$ is given through: 
   \begin{equation*}
       5 = 2 \cdot 3^0 + 1 \cdot 3^1,
   \end{equation*}
   such that $a_1 = 2, a_2 = 1$. Accordingly, we obtain:
   \begin{equation*}
       \phi_3(5) = \frac{2}{3^1} + \frac{1}{3^2} = \frac{6}{9} + \frac{1}{9} = \frac{7}{9} \approx 0.778.
   \end{equation*}
   
   END SOLUTION 
   
With this knowledge, we implement the function [`van_der_corput_sequence`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.
Once, the implementation has been completed, we check its validity for $b=10$ and $n_\mathrm{max}=20$.

In [ ]:
from e2ml.experimentation import van_der_corput_sequence
exp_sequence = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 
                         0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91,
                         0.02])
np.testing.assert_array_almost_equal(van_der_corput_sequence(20, 10), exp_sequence)

Multi-dimensional Halton sequences are constructed by van der Corput sequences that use coprime numbers as its bases. For this purpose, we implement the function [`primes_from_2_to`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. This function generates prime numbers from $2$ to $n_\mathrm{max}$. We check the function's validity for the prime numbers until $50$.

In [ ]:
from e2ml.experimentation import primes_from_2_to
exp_prime_numbers = np.array([2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])
np.testing.assert_array_equal(primes_from_2_to(50), exp_prime_numbers)

Given the van der Corput sequence and primer numbers generator, we can now implement a multi-dimensional Halton sequence by using for each dimension one van der Corput sequence with a unique prime number as basis, i.e., each dimension must use a different prime number.
We implement the function [`halton_unit`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage. It generates samples $\mathbf{x} \in [0, 1]^D$.

In [ ]:
from e2ml.experimentation import halton_unit
# Generate 200 two-dimensional samples with the Halton sequence.
X = halton_unit(200, 2) # <-- SOLUTION

# Plot generated samples.
# BEGIN SOLUTION
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()
# END SOLUTION

In many experiments, we want to generate samples whose levels lie in specific intervals for each dimension.

#### Question:
1. (c) How to scale the values $x_1, \dots, x_N \in [0, 1]$ such that all of them lie in the interval $[a, b]$ with $a,b \in \mathbb{R}$.

   BEGIN SOLUTION
   
   The transformation is defined as:
   \begin{equation*}
       z_n = x_n \cdot (b - a) + a.
   \end{equation*}
   END SOLUTION
   
We extend the function [`halton_unit`](../e2ml/experimentation/_halton.py) by allowing to define boundaries for each dimension's interval through scaling. Therefore, we implement the function [`halton`](../e2ml/experimentation/_halton.py) in the [`e2ml.experimentation`](../e2ml/experimentation) subpackage.

In [ ]:
from e2ml.experimentation import halton

# Generate 200 two-dimensional samples with the Halton sequence
# in the hypercube [-2, 2] x [1, 5].
bounds = np.array([[-2, 2], [1, 5]]) # <-- SOLUTION
X = halton(200, 2, bounds) # <-- SOLUTION

# Plot generated samples.
# BEGIN SOLUTION
plt.scatter(X[:, 0], X[:, 1])
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.show()
# END SOLUTION

### **2. Comparison of Space Filling Techniques** <a class="anchor" id="comparison"></a>

In this section, we compare the Halton sequence sampling to uniform sampling. First, we compare them just visually.
Therefore, we plot 200 samples generated through a uniform distribution and compare them to 200 samples generated according to the Halton sequence in the two-dimensional unit cube.

In [ ]:
# Visual comparison of uniform sampling and Halton sequence sampling.
# BEGIN SOLUTION
X_halton = halton(200, 2) 
X_random = np.random.uniform(size=(200, 2))
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax[0].set_title('Halton sequence sampling', fontsize=15)
ax[0].scatter(X_halton[:, 0], X_halton[:, 1])
ax[0].set_xlabel('$x_1$', fontsize=15)
ax[0].set_ylabel('$x_2$', fontsize=15)
ax[1].set_title('Uniform sampling', fontsize=15)
ax[1].scatter(X_random[:, 0], X_random[:, 1])
ax[1].set_xlabel('$x_1$', fontsize=15)
ax[1].set_ylabel('$x_2$', fontsize=15)
plt.show()
# END SOLUTION

For a quantitative comparison, we want to approximate the integral of a cosine-shaped function in the interval $[-\pi, +\pi]$:

\begin{align*}
    f(x) &= \cos(x) + 1, \\
    F(x) &= \sin(x) + x + C.
\end{align*}

Accordingly, the definite integral in the interval $[-\pi, +\pi]$ is given through:

BEGIN SOLUTION

\begin{equation*}
    F(\pi) - F(-\pi) = 2\pi.
\end{equation*}

END SOLUTION

#### Question:
1. (a) Imagine, we can only generate samples of the form $(x_1, x_2)^\mathrm{T}$ with $x_1 \in [-\pi, +\pi], x_2 \in [0, 2]$ and can only ask for the information whether $x_2 \in [0, f(x_1)]$ or $x_2 \in (f(x_1), 2]$ is true. How can we approximate the above definite integral through sampling and making use of this information?

   BEGIN SOLUTION
   
   We can generate samples in the space $[-\pi, \pi] \times [0, 2]$, which has the area $4\pi$ and compute the proportion $\rho \in [0, 1]$ of samples fulfilling $x_2 \in (f(x_1), 2]$. Accordingly, the approximated area is $\rho \cdot 4\pi$.
   
   END SOLUTION
   
In the following, we compare the error of uniform sampling and Halton sequence sampling to approximate the above definite integral for different numbers of samples.

In [ ]:
def compute_approximation_error(X):
    """
    Computes mean squared error for the definite integral of
    function f(x) = cos(x) + 1 in the interval [-pi, pi].
    
    Parameters
    ----------
    X : numpy.ndarray of shape (n_samples, 2)
        Samples with `X[:, 0]` in [-pi, pi] and `X[:, 1]` in [0, 2].
        
    Returns
    -------
    error : float
        Mean squared error between approximated and true value of
        the definite integral.
    """
    # BEGIN SOLUTION
    f = np.cos(X[:, 0]) + 1
    rho = np.mean(X[:, 1] <= f) 
    error = (rho * 4 * np.pi - 2 * np.pi)**2
    return error
    # END SOLUTION

# Numbers of samples to be tested.
n_samples_list = np.linspace(100, 10000, 100, dtype=int)

# Compute approximation errors for Halton sequence sampling.
# BEGIN SOLUTION
errors_halton = np.zeros_like(n_samples_list)
for i, n_samples in enumerate(n_samples_list):
    X_halton = halton(int(n_samples), 2, bounds=[[-np.pi, np.pi], [0, 2]])
    errors_halton[i] = compute_approximation_error(X_halton)
# END SOLUTION

# Compute approximation errors for uniform sampling.
# BEGIN SOLUTION
n_trials = 100
errors_random = np.zeros((len(n_samples_list), n_trials))
for j in range(n_trials):
    for i, n_samples in enumerate(n_samples_list):
        X_random = np.random.uniform(size=(n_samples, 2))
        X_random[:, 0] = X_random[:, 0] * (np.pi - (-np.pi)) - np.pi
        X_random[:, 1] = X_random[:, 1] * (2 - 0) - 0
        errors_random[i, j] = compute_approximation_error(X_random)
errors_random_mean = errors_random.mean(axis=1)
# END SOLUTION

# Plot computed approximation errors along numbers of generated samples.
# BEGIN SOLUTION
plt.figure()    
plt.plot(n_samples_list, errors_halton, label='Halton sequence sampling')
plt.plot(n_samples_list, errors_random_mean, label='Uniform sampling')
plt.xlabel('# samples', fontsize=15)
plt.ylabel('approximation error', fontsize=15)
plt.legend(prop={'size': 15})
plt.show()
# END SOLUTION